In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import time
import requests
from datetime import datetime, timedelta
from shapely.geometry import Point
from busSim import BusSim
from graph import Graph
from geopy.distance import geodesic

In [2]:
df = gpd.read_file("zip://DataMerge/block_group_2019.zip")
df["centroid"] = df.centroid
gdf = df[["GEOID10","centroid"]]
gdf = gdf.rename(columns={"centroid":"geometry"})

c:\pathon\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  


In [3]:
gdf

,GEOID10,geometry
0,550250014031,POINT (-89.44025 43.02010)
1,550250020003,POINT (-89.34386 43.09155)
2,550250021002,POINT (-89.34670 43.10461)
3,550250021004,POINT (-89.35983 43.09928)
4,550250021003,POINT (-89.35331 43.10229)
...,...,...
195,550250105021,POINT (-89.26710 43.02063)
196,550250102001,POINT (-89.37188 43.11287)
197,550250018042,POINT (-89.36598 43.09629)
198,550259917020,POINT (-89.41980 43.10540)


In [4]:
test = gdf.iloc[:15,:]

In [5]:
test

,GEOID10,geometry
0,550250014031,POINT (-89.44025 43.02010)
1,550250020003,POINT (-89.34386 43.09155)
2,550250021002,POINT (-89.34670 43.10461)
3,550250021004,POINT (-89.35983 43.09928)
4,550250021003,POINT (-89.35331 43.10229)
5,550250022002,POINT (-89.35929 43.11809)
6,550250022001,POINT (-89.35974 43.12561)
7,550250022003,POINT (-89.36695 43.12050)
8,550250023012,POINT (-89.37705 43.13004)
9,550250023011,POINT (-89.38199 43.13751)


In [6]:
def lonlat(df):
    df_lon=[]
    df_lat = []
    for j in range(len(df)):
        delhi_lon,delhi_lat = df['geometry'].iloc[j].x, df['geometry'].iloc[j].y
        df_lon.append(delhi_lon)
        df_lat.append(delhi_lat)
    df['lon']=df_lon
    df['lat']=df_lat
    return df


In [19]:
test

,GEOID10,geometry,lon,lat,location
0,550250014031,POINT (-89.44025 43.02010),-89.440253,43.020104,"(-89.44025297823605, 43.02010411472464)"
1,550250020003,POINT (-89.34386 43.09155),-89.343862,43.091551,"(-89.34386248640314, 43.091551144245436)"
2,550250021002,POINT (-89.34670 43.10461),-89.346698,43.104609,"(-89.34669792629813, 43.104609250272986)"
3,550250021004,POINT (-89.35983 43.09928),-89.359829,43.099275,"(-89.35982916386286, 43.099275386480954)"
4,550250021003,POINT (-89.35331 43.10229),-89.353314,43.102288,"(-89.35331357728143, 43.10228839448924)"
5,550250022002,POINT (-89.35929 43.11809),-89.359294,43.118089,"(-89.35929401071601, 43.118088875037714)"
6,550250022001,POINT (-89.35974 43.12561),-89.359742,43.125608,"(-89.35974209988959, 43.12560787675973)"
7,550250022003,POINT (-89.36695 43.12050),-89.366946,43.120502,"(-89.3669458172327, 43.120501560092116)"
8,550250023012,POINT (-89.37705 43.13004),-89.377055,43.130041,"(-89.3770547323741, 43.13004085470157)"
9,550250023011,POINT (-89.38199 43.13751),-89.381986,43.137514,"(-89.38198593746667, 43.137513785260715)"


In [9]:
lat_max = test["lat"].max()
lat_min = test["lat"].min()
lon_max = test["lon"].max()
lon_min = test["lon"].min()

In [26]:
df = pd.DataFrame(
    {'edge': ['leftHigh', 'righHigh', 'LeftLow', 'rightLow'],
     'Latitude': [lat_max, lat_max, lat_min, lat_min],
     'Longitude': [lon_max, lon_min, lon_max, lon_min]})

gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))

In [27]:
gdf

,edge,Latitude,Longitude,geometry
0,leftHigh,43.137636,-89.343862,POINT (-89.34386 43.13764)
1,righHigh,43.137636,-89.516538,POINT (-89.51654 43.13764)
2,LeftLow,43.020104,-89.343862,POINT (-89.34386 43.02010)
3,rightLow,43.020104,-89.516538,POINT (-89.51654 43.02010)


In [12]:
def outpoint(df):
    df_distance=[]
    for j in range(len(df)):
        delhi_1 = (df['geometry'].iloc[j].x, df['geometry'].iloc[j].y)
        df_distance.append(delhi_1)
    df['location']=df_distance
    return df

In [30]:
gdf = outpoint(gdf)
outpoint(test)

c:\pathon\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,GEOID10,geometry,lon,lat,location
0,550250014031,POINT (-89.44025 43.02010),-89.440253,43.020104,"(-89.44025297823605, 43.02010411472464)"
1,550250020003,POINT (-89.34386 43.09155),-89.343862,43.091551,"(-89.34386248640314, 43.091551144245436)"
2,550250021002,POINT (-89.34670 43.10461),-89.346698,43.104609,"(-89.34669792629813, 43.104609250272986)"
3,550250021004,POINT (-89.35983 43.09928),-89.359829,43.099275,"(-89.35982916386286, 43.099275386480954)"
4,550250021003,POINT (-89.35331 43.10229),-89.353314,43.102288,"(-89.35331357728143, 43.10228839448924)"
5,550250022002,POINT (-89.35929 43.11809),-89.359294,43.118089,"(-89.35929401071601, 43.118088875037714)"
6,550250022001,POINT (-89.35974 43.12561),-89.359742,43.125608,"(-89.35974209988959, 43.12560787675973)"
7,550250022003,POINT (-89.36695 43.12050),-89.366946,43.120502,"(-89.3669458172327, 43.120501560092116)"
8,550250023012,POINT (-89.37705 43.13004),-89.377055,43.130041,"(-89.3770547323741, 43.13004085470157)"
9,550250023011,POINT (-89.38199 43.13751),-89.381986,43.137514,"(-89.38198593746667, 43.137513785260715)"


In [62]:
result = {}
for gdfidx,gdfrow in gdf.iterrows():
    smallest_new={}
    for testidx,testrow in test.iterrows():
        distance=geodesic(gdfrow['location'],testrow['location']).miles
        smallest_new[testidx]=distance
        
    
    #test = test.sort_values("distance")[:3]
    result[gdfrow["edge"]] = pd.DataFrame.from_dict(smallest_new,orient='index',columns=["distance"]).sort_values("distance")


In [63]:
result


{'leftHigh':      distance
 1    0.036627
 2    0.198524
 4    0.656530
 5    1.071109
 6    1.102139
 3    1.108550
 7    1.602116
 8    2.303660
 9    2.645895
 10   3.920728
 0    6.690379
 11   8.477700
 13  11.490396
 12  11.541398
 14  11.984518, 'righHigh':      distance
 14   0.068050
 12   0.446162
 13   0.497567
 11   3.506930
 0    5.294952
 10   8.063529
 9    9.338362
 8    9.680605
 7   10.382202
 3   10.876146
 6   10.882161
 5   10.913265
 4   11.328344
 2   11.787489
 1   11.984297, 'LeftLow':      distance
 1    0.056784
 2    0.207887
 4    0.659135
 5    1.073760
 6    1.105206
 3    1.109882
 7    1.603976
 8    2.305225
 9    2.647444
 10   3.921744
 0    6.689822
 11   8.477644
 13  11.490197
 12  11.541249
 14  11.984256, 'rightLow':      distance
 14   0.000779
 12   0.443328
 13   0.494175
 11   3.506831
 0    5.294434
 10   8.063893
 9    9.338685
 8    9.680879
 7   10.382414
 3   10.876246
 6   10.882390
 5   10.913457
 4   11.328456
 2   11.787608
 1   11.

In [49]:
result["leftHigh"]-result["LeftLow"]

,distance
1,-0.020157
2,-0.009363
4,-0.002605
5,-0.002651
6,-0.003068
3,-0.001333
7,-0.001860
8,-0.001565
9,-0.001549
10,-0.001017
